In [1]:
# This notebook makes setolabo samples as test samples. 

#Kagawa samples preprocessing

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#dropping empty rows and cleaning up

# Load your CSV file into a pandas DataFrame
file_path = 'E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/Colorectal/Colorectal_SRP411850_PRJNA909776.csv'
df = pd.read_csv(file_path)

# Keep the 'id' column separate
id_column = df.iloc[:, 0]  # First column assumed to be the 'id'

# Process the rest of the columns
data = df.iloc[:, 1:]  # Exclude the 'id' column for calculations

# Ensure missing values are correctly recognized (e.g., empty strings or unusual placeholders)
data.replace(["", " ", "NA", "N/A", "-"], pd.NA, inplace=True)

# Drop rows where more than 50% of the values are missing
threshold = len(data.columns) / 2  # 50% of the columns (excluding 'id')
data = data.dropna(thresh=threshold)  # Keep rows with at least 50% non-NaN values

# Fill missing values in the remaining rows with the row's median
data = data.apply(lambda row: row.fillna(row.median()), axis=1)

# Reattach the 'id' column to the processed data
cleaned_df = pd.concat([id_column.loc[data.index], data], axis=1)

# Save the cleaned DataFrame back to a CSV file (optional)
cleaned_df.to_csv("E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/Colorectal/2Colorectal_SRP411850_PRJNA909776.csv", index=False)

print("Rows processed: \n", cleaned_df)



Rows processed: 
               miRNA  advanced adenoma plasma EV 1  \
0     hsa-let-7a-5p                       20352.0   
1     hsa-let-7c-5p                        1208.0   
2     hsa-let-7e-5p                         439.0   
3     hsa-let-7f-5p                        2955.0   
4     hsa-let-7g-5p                        2505.0   
..              ...                           ...   
877  hsa-miR-942-5p                         266.0   
880   hsa-miR-95-5p                          19.0   
881   hsa-miR-96-5p                          88.0   
882   hsa-miR-98-5p                         131.0   
886  hsa-miR-99a-5p                        7043.0   

     advanced adenoma plasma EV 2  advanced adenoma plasma EV 3  \
0                         50386.0                        5512.0   
1                          4332.0                         399.0   
2                           646.0                         194.0   
3                          8517.0                         761.0   
4         

In [7]:
import pandas as pd

# Load data
df_GRE = pd.read_csv('E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/Colorectal/2Colorectal_SRP411850_PRJNA909776.csv')
df_seto = pd.read_csv('E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/expression_count_test_20240807.csv')

# Remove duplicates from df_GRE based on 'miRNA'
df_GRE = df_GRE.drop_duplicates(subset=['miRNA'], keep='first')

# Create labels for df_GRE (case = 0, control = 1)
columns = df_GRE.columns.tolist()
labels_GRE = []
for item in columns:
    if item.startswith('colorectal'):
        labels_GRE.append(0)
    elif item.startswith('healthy'):
        labels_GRE.append(1)
    elif item.startswith('advanced'):
        labels_GRE.append(2)    
    # elif item.startswith('ColonC'):
    #     labels_GRE.append(3)   
    # elif item.startswith('GallbladderC'):
    #     labels_GRE.append(4)          
    else:
        pass

# Set labels for df_seto as 0 (since all are cancerous)
labels_seto = [0 for _ in range(5)]

# Filter both datasets based on common 'miRNA' / 'gene_id'
features = df_GRE['miRNA'].tolist()
df_seto = df_seto[df_seto['gene_id'].isin(features)]
features = df_seto['gene_id']
df_GRE = df_GRE[df_GRE['miRNA'].isin(features)]

# Sort both DataFrames by 'miRNA' / 'gene_id' to align them
df_GRE = df_GRE.sort_values(by='miRNA')
df_seto = df_seto.sort_values(by='gene_id')

# Transpose the DataFrames and reset their index
df_GRE = df_GRE.T
df_seto = df_seto.T
df_GRE.reset_index(drop=True, inplace=True)
df_seto.reset_index(drop=True, inplace=True)

# Update the column names after transposing
df_seto.columns = df_seto.iloc[0].tolist()
df_seto = df_seto[1:]
df_GRE.columns = df_GRE.iloc[0].tolist()
df_GRE = df_GRE[1:]

# Assign labels to the GRE and seto datasets
df_GRE['labels'] = labels_GRE
df_seto['labels'] = labels_seto

# No train-test split, df_GRE is the full training set and df_seto is the testing set
df_train = df_GRE.iloc[:, :-1]  # Features of the full training set
df_test = df_seto.iloc[:, :-1]  # Features of the full testing set

# Create labels_train and labels_test
labels_train = df_GRE['labels'].tolist()
labels_test = df_seto['labels'].tolist()


In [8]:
# Define NGS version as a variable
folder_path = "E:/MedBank/head_model/NGS/data_shared_by_Takeshi-san/model_ready_folders/setolabo_samples_as_test/Colorectal_PRJNA909776"  # You can change this value as needed

# Paths for train and test feature vectors
file_path = f"{folder_path}/train/feature_vectors.csv"
df_train.to_csv(file_path, index=False, header=False)

file_path = f"{folder_path}/test/feature_vectors.csv"
df_test.to_csv(file_path, index=False, header=False)

##--------------------------------------------------------

# Paths for train and test labels
file_path = f"{folder_path}/train/labels.txt"
with open(file_path, 'w') as file:
    for label in labels_train:
        file.write(f"{label}\n")

file_path = f"{folder_path}/test/labels.txt"
with open(file_path, 'w') as file:
    for label in labels_test:
        file.write(f"{label}\n")

##--------------------------------------------------------

# Path for feature names
file_path = f"{folder_path}/feature_names.txt"
with open(file_path, 'w') as file:
    for label in features:
        file.write(f"{label}\n")


In [1]:
text = 'case/sample_analysis/ERR372264/ERR372264_sort.bam'
if text.startswith('case'):
    print(True)

True
